# LangGraph と AgentCore Memory Tool（短期メモリ）

## はじめに
このノートブックでは、LangGraph フレームワークを使用した会話型 AI エージェントに Amazon Bedrock AgentCore Memory 機能を統合する方法を紹介します。単一の会話セッション内での**短期メモリ**の保持に焦点を当て、明示的なコンテキスト管理なしで会話の早い段階の情報をエージェントが想起できるようにします。


## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | パーソナルフィットネス                                                          |
| エージェントフレームワーク | LangGraph                                                                       |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、LangGraph、ツール経由のメモリ取得                          |
| 難易度              | 初級                                                                            |

学習内容：
- 短期メモリ用の AgentCore Memory を使用したメモリストアの作成
- LangGraph を使用した構造化メモリワークフローを持つエージェントの作成
- 会話履歴取得用のメモリツールの実装
- 単一セッション内での文脈情報へのアクセスと活用
- 効果的なメモリ想起による会話体験の向上


### シナリオのコンテキスト

この例では、会話全体を通じてワークアウトの詳細、フィットネス目標、身体的制限、エクササイズの好みを記憶できる「**パーソナルフィットネスコーチ**」を作成します。このアシスタントは、効果的な短期メモリ管理により、ユーザーが繰り返し情報を述べる必要なく、より自然でパーソナライズされたフィットネスコーチング体験を可能にする方法を実演します。


## アーキテクチャ
<div style="text-align:left">
    <img src="images/architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory の適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境のセットアップから始めましょう！

## ステップ 1: 環境のセットアップ
このノートブックを動作させるために必要なすべてのライブラリのインポートとクライアントの定義から始めましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

Amazon Bedrock モデルと AgentCore に適切な権限を持つリージョンとロールを定義します

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

### 統合の仕組み

LangGraph と AgentCore Memory の統合には以下が含まれます：

1. AgentCore Memory を使用して会話を短期メモリに保存
2. メモリ操作を管理する LangGraph の構造化ワークフロー

このアプローチは、メモリ管理と推論を分離し、よりクリーンで保守性の高いエージェントアーキテクチャを実現します。

## ステップ 2: メモリの作成
このセクションでは、AgentCore Memory SDK を使用してメモリストアを作成します。このメモリストアにより、エージェントは会話から情報を保持できます。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "FitnessCoach"
memory_id = None

In [ ]:
try:
    print("Creating Memory...")
    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # This name is unique across all memories in this account
        description="Fitness Coach Agent",      # Human-readable description
        strategies=[],                          # No memory strategies for short-term memory
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    logger.info(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## ステップ 3: LangGraph エージェントの作成
LangGraph でエージェントを作成するために必要なすべてのライブラリをインポートしましょう。

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_aws import ChatBedrock

### LangGraph エージェントの実装

メモリツールを組み込んで LangGraph でエージェントを作成しましょう：

In [ ]:
def create_agent(client, memory_id, actor_id, session_id):
    """Create and configure the LangGraph agent"""
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    @tool
    def list_events():
        """Tool used when needed to retrieve recent information""" 
        events = client.list_events(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                max_results=10
            )
        return events
        
    
    # Bind tools to the LLM
    tools = [list_events]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = """You are the Personal Fitness Coach, a sophisticated fitness guidance assistant.
                        PURPOSE:
                        - Help users develop workout routines based on their fitness goals
                        - Remember user's exercise preferences, limitations, and progress
                        - Provide personalized fitness recommendations and training plans
                        MEMORY CAPABILITIES:
                        - You have access to recent events with the list_events tool
                        """
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        raw_messages = state["messages"]
    
        # Remove any existing system messages to avoid duplicates or misplacement
        non_system_messages = [msg for msg in raw_messages if not isinstance(msg, SystemMessage)]
    
        # Always ensure SystemMessage is first
        messages = [SystemMessage(content=system_message)] + non_system_messages
    
        latest_user_message = next((msg.content for msg in reversed(messages) if isinstance(msg, HumanMessage)), None)
    
        # Get response from model with tools bound
        response = llm_with_tools.invoke(messages)
    
        # Save conversation if applicable
        if latest_user_message and response.content.strip():  # Check that response has content
            conversation = [
                (latest_user_message, "USER"),
                (response.content, "ASSISTANT")
            ]
            
            # Validate that all message texts are non-empty
            if all(msg[0].strip() for msg in conversation):  # Ensure no empty messages
                try:
                    client.create_event(
                        memory_id=memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=conversation
                    )
                except Exception as e:
                    print(f"会話の保存エラー: {str(e)}")
        
        # Append response to full message history
        return {"messages": raw_messages + [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

### エージェント呼び出し用のラッパーの作成

エージェントを呼び出すためのシンプルなラッパーを作成しましょう：

In [ ]:
def langgraph_bedrock(payload, agent):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

## ステップ 4: LangGraph エージェントの実行
AgentCore Memory 統合でエージェントを実行できるようになりました。

In [ ]:
# Create unique actor and session IDs for this conversation
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"workout-{datetime.now().strftime('%Y%m%d%H%M%S')}"

In [ ]:
# Create the agent with AgentCore Memory integration
agent = create_agent(client, memory_id, actor_id, session_id)

#### おめでとうございます！エージェントの準備ができました！

### エージェントをテストしましょう

エージェントと対話してメモリ機能をテストしましょう：

In [ ]:
response = langgraph_bedrock({"prompt": "Hello! This is my first day, I need a workout routine."}, agent)
print(f"エージェント: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "I want to build muscle, looking for a biceps routine. I have some lower back problems."}, agent)
print(f"エージェント: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "Can you give me three exercises with number of reps?"}, agent)
print(f"エージェント: {response}\n")

### メモリ永続化のテスト

AgentCore Memory 統合の威力を真に実証するために、新しいエージェントインスタンスを作成し、以前の会話を想起できるか確認しましょう：

In [ ]:
# Create a new agent instance (simulating a new session)
new_agent = create_agent(client, memory_id, actor_id, session_id)

# Test if the new agent remembers our preferences
response = langgraph_bedrock({
    "prompt": "Hello again! Can you remind me about my last workout session?"
}, new_agent)

print("新しいエージェントセッション:\n")
print(f"エージェント: {response}")

## まとめ

このノートブックでは、以下を実演しました：

1. AI エージェント用の AgentCore Memory リソースの作成方法
2. メモリ統合を持つ LangGraph ワークフローの構築
3. 会話履歴取得用のメモリツールの実装
4. 必要に応じてインテリジェントにメモリを使用するエージェントの作成
5. エージェントインスタンス間でのメモリ永続化のテスト

この統合は、構造化されたワークフロー（LangGraph）と堅牢なメモリシステム（AgentCore Memory）を組み合わせることで、よりインテリジェントでコンテキストを意識した AI エージェントを作成する力を示しています。

ここで実演したアプローチは、マルチエージェントシステム、抽出ストラテジーを使用した長期メモリ、会話コンテキストに基づく特殊なメモリ取得など、より複雑なユースケースに拡張できます。

## クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(memory_id = memory_id, max_wait = 300, poll_interval =10)